<a href="https://colab.research.google.com/github/skillnerve/DataScience-Projects/blob/main/Dog_Breed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
import keras.backend as K
from tensorflow.keras.applications import vgg16 
from tensorflow.keras.applications import resnet50
from tensorflow.keras.applications import xception
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.applications import inception_resnet_v2
from tqdm import tqdm

# Data

In [ ]:
train_csv=pd.read_csv('../input/dog-breed-identification/labels.csv')

In [ ]:
train_csv.head()

In [ ]:
breed=list(set(train_csv['breed']))
q=0
for i in breed:
    train_csv['breed'][train_csv.breed==i]=q
    q+=1

In [ ]:
print(breed)

# Data Split

In [ ]:
x_train,x_test=train_test_split(train_csv['id'],test_size=0.20)
x_train,x_cv  =train_test_split(x_train,test_size=0.18)

# Data Generator

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,x_in,df=train_csv,batch_size=10,n_classes=120):
        self.batch_size = batch_size
        self.x_in = x_in
        self.df   = df
        self.n_classes = n_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.x_in.shape[0] / self.batch_size))

    def __getitem__(self, index):
        x_VGG,x_INRE,x_IN,x_XC, y = self.__data_generation(self.x_in[index*self.batch_size:(index+1)*self.batch_size])
        return (x_VGG,x_INRE,x_IN,x_XC), y

    def __data_generation(self, ini):
        x_VGG  = np.empty((self.batch_size,224,224,3))
        x_INRE = np.empty((self.batch_size,299,299,3))
        x_IN   = np.empty((self.batch_size,299,299,3))
        x_RE   = np.empty((self.batch_size,224,224,3))
        x_XC   = np.empty((self.batch_size,299,299,3))
        y = np.empty((self.batch_size))
        
        for i in range(ini.shape[0]):
            t   = cv2.imread("../input/dog-breed-identification/train/"+str(ini.values[i])+'.jpg')
            x_VGG[i]  = vgg16.preprocess_input(cv2.resize(t,(224,224)))
            x_INRE[i] = inception_resnet_v2.preprocess_input(cv2.resize(t,(299,299)))
            x_IN[i]   = inception_v3.preprocess_input(cv2.resize(t,(299,299)))
            x_XC[i]   = xception.preprocess_input(cv2.resize(t,(299,299)))
            y[i]      = self.df[self.df.id==str(ini.values[i])]['breed'].values[0]
        return x_VGG,x_INRE,x_IN,x_XC, tf.keras.utils.to_categorical(y, num_classes=self.n_classes)

train_generator=DataGenerator(x_train)
cv_generator   =DataGenerator(x_cv)

In [ ]:
class Test_DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,x_in,batch_size=1,n_classes=120):
        self.batch_size = batch_size
        self.x_in = x_in
        self.n_classes = n_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.x_in.shape[0] / self.batch_size))

    def __getitem__(self, index):
        x_VGG,x_INRE,x_IN,x_XC = self.__data_generation(self.x_in[index*self.batch_size:(index+1)*self.batch_size])
        return x_VGG,x_INRE,x_IN,x_XC

    def __data_generation(self, ini):
        x_VGG  = np.empty((self.batch_size,224,224,3))
        x_INRE = np.empty((self.batch_size,299,299,3))
        x_IN   = np.empty((self.batch_size,299,299,3))
        x_XC   = np.empty((self.batch_size,299,299,3))
        for i in range(ini.shape[0]):
            t   = cv2.imread("../input/dog-breed-identification/train/"+str(ini.values[i])+'.jpg')
            x_VGG[i]  = vgg16.preprocess_input(cv2.resize(t,(224,224)))
            x_INRE[i] = inception_resnet_v2.preprocess_input(cv2.resize(t,(299,299)))
            x_IN[i]   = inception_v3.preprocess_input(cv2.resize(t,(299,299)))
            x_XC[i]   = xception.preprocess_input(cv2.resize(t,(299,299)))
        return x_VGG,x_INRE,x_IN,x_XC

test_generator=Test_DataGenerator(x_test)

# Callbacks

In [ ]:
class CustomCallBacks(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        self.accuracy     ={'accuracy':[]}
        self.val_accuracy ={'accuracy':[]}
        self.epoch    =None
        self.b        =0
        self.c        =0
        self.max_t    =None 
        
    def on_epoch_begin(self, epoch, logs={}):
        self.epoch=epoch
        if self.epoch>=3 and self.epoch%3==0:
            lr=tf.keras.backend.get_value(self.model.optimizer.lr)
            tf.keras.backend.set_value(self.model.optimizer.lr,lr*0.5)

    def on_epoch_end(self, epoch, logs={}):
        print('{} ended'.format(epoch))
        self.accuracy['accuracy'].append(logs.get('accuracy'))
        self.val_accuracy['accuracy'].append(logs.get('val_accuracy'))
        if self.epoch==0:
            self.model.save_weights('dog_breed.h5')
            self.max_t=self.val_accuracy['accuracy'][-1]
        if self.epoch>=1:
            if self.val_accuracy['accuracy'][-1]>=self.max_t and self.val_accuracy['accuracy'][-1]-self.accuracy['accuracy'][-1]>=-0.03:
                self.max_t=self.val_accuracy['accuracy'][-1]
                self.model.save_weights('dog_breed.h5')
                print('*'*5)
                print("weight updated")
                print('*'*5)
            if self.val_accuracy['accuracy'][-1]<self.accuracy['accuracy'][-1]:
                self.b+=1
                if self.b==2:
                    self.model.stop_training = True
cb=CustomCallBacks()

# Model

In [ ]:
base_model_1 = vgg16.VGG16(weights='../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224,224,3))
base_model_2 = inception_resnet_v2.InceptionResNetV2(weights='../input/keras-pretrained-models/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(299,299,3))
base_model_3 = inception_v3.InceptionV3(weights='../input/keras-pretrained-models/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(299,299,3))
base_model_4 = xception.Xception(weights='../input/keras-pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(299,299,3))

In [ ]:
mod=[base_model_1,base_model_2,base_model_3,base_model_4]
for i in tqdm(range(4)):
    for layer in mod[i].layers:
        layer._name+=str(i)

In [ ]:
base_model_1.trainable = False
base_model_2.trainable = False
base_model_3.trainable = False
base_model_4.trainable = False

In [ ]:
x1=tf.keras.layers.GlobalAveragePooling2D()(base_model_1.output)
x2=tf.keras.layers.GlobalAveragePooling2D()(base_model_2.output)
x3=tf.keras.layers.GlobalAveragePooling2D()(base_model_3.output)
x4=tf.keras.layers.GlobalAveragePooling2D()(base_model_4.output)

x = tf.keras.layers.Concatenate(axis=-1)([x1, x2, x3, x4])
x = tf.keras.layers.Dense(256,activation='linear')(x)
x = tf.keras.layers.Dropout(.5)(x)
outputs = tf.keras.layers.Dense(120, activation='softmax')(x)
model = tf.keras.Model(inputs=[base_model_1.input,base_model_2.input,base_model_3.input,base_model_4.input], outputs=outputs)

In [ ]:
tf.config.run_functions_eagerly(True)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,loss='CategoricalCrossentropy',metrics=['accuracy'])

In [ ]:
model.fit_generator(train_generator,validation_data=cv_generator,epochs=15,callbacks=[cb])